In [1]:
import pandas as pd
import re
from pathlib import Path

RAW = "../data/raw/ons_cpi_mm23.csv"
OUT = "../data/processed/cpi_personal_care.csv"
META_OUT = "../data/processed/cpi_personal_care_meta.json"

# 1) Load once
raw = pd.read_csv(RAW)

# 2) CDID map (row 0) for documentation
cdid_map = {raw.columns[i]: raw.iloc[0, i] for i in range(1, raw.shape[1])}

# 2a) Persist the column-title → CDID mapping (handy for report/appendix)
(
    pd.Series(cdid_map, name="CDID")
      .to_frame()
      .rename_axis("series_title")
      .reset_index()
      .to_csv("../data/processed/cpi_cdid_mapping.csv", index=False)
)

# 3) Remove the top metadata rows so we only have time labels + data
meta_rows = {"CDID", "PreUnit", "Unit", "Release Date", "Next release"}
df = raw[~raw["Title"].isin(meta_rows)].copy()

# 4) Keep monthly rows only
month_names = r"(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|June|July|August|September|October|November|December)"
patterns = [
    rf"^{month_names}\s+\d{{4}}$",      # "Jan 1997"
    rf"^\d{{4}}\s+{month_names}$",      # "1997 Jan"
    r"^\d{4}-(0[1-9]|1[0-2])$",         # "1997-01"
    r"^\d{4}-(0[1-9]|1[0-2])-\d{2}$",   # "1997-01-01"
]

def looks_monthly(s: str) -> bool:
    s = str(s).strip()
    return any(re.search(p, s, flags=re.IGNORECASE) for p in patterns)

monthly = df[df["Title"].astype(str).apply(looks_monthly)].copy()

def parse_mixed_date(s: str):
    s = str(s).strip().replace(".", "")
    for fmt in ["%b %Y", "%B %Y", "%Y %b", "%Y %B", "%Y-%m", "%Y-%m-%d", "%d-%m-%Y"]:
        try:
            return pd.to_datetime(s, format=fmt)
        except Exception:
            pass
    return pd.to_datetime(s, errors="coerce")

monthly["date"] = monthly["Title"].apply(parse_mixed_date)
monthly = monthly[monthly["date"].notna()].sort_values("date").reset_index(drop=True)

# 5) Select target series
# --- Option A (current, works): by title ---
# TARGET_COL = "CPI INDEX 12.1 : PERSONAL CARE 2015=100"

# --- Option B (recommended, future-proof): by CDID ---
TARGET_CDID = "D7CY"
TARGET_COL = next(col for col, cd in cdid_map.items() if cd == TARGET_CDID)
print("Selected by CDID:", TARGET_CDID, "->", TARGET_COL)

# 6) Build tidy series
series = monthly[["date", TARGET_COL]].copy()
series.rename(columns={TARGET_COL: "cpi_index"}, inplace=True)
series["cpi_index"] = pd.to_numeric(series["cpi_index"], errors="coerce")
series = series.dropna(subset=["cpi_index"]).sort_values("date").reset_index(drop=True)

# 7) Sanity checks
yr2015 = series[series["date"].dt.year == 2015]["cpi_index"].median()
assert 90 <= yr2015 <= 110, f"2015 base sanity failed; got median={yr2015}"
assert not series["date"].duplicated().any(), "Duplicate dates found."
assert series["date"].is_monotonic_increasing, "Dates are not sorted."

# 8) Derived metrics
series["pct_change_mom"] = series["cpi_index"].pct_change() * 100
series["pct_change_yoy"] = series["cpi_index"].pct_change(12) * 100

# 9) Save processed data
Path(OUT).parent.mkdir(parents=True, exist_ok=True)
series.to_csv(OUT, index=False)

# 10) Save minimal metadata for this series (CDID, Unit, etc.)
meta = {
    "source_file": RAW,
    "series_title": TARGET_COL,
    "cdid": cdid_map.get(TARGET_COL, None),
    "unit": str(raw.loc[2, TARGET_COL]) if TARGET_COL in raw.columns else None,
    "release_date": str(raw.loc[3, TARGET_COL]) if TARGET_COL in raw.columns else None,
    "base_year": "2015=100",
    "rows": int(series.shape[0]),
    "date_min": str(series["date"].min().date()),
    "date_max": str(series["date"].max().date()),
}
pd.Series(meta).to_json(META_OUT, indent=2)

OUT, series.shape, meta

C:\Users\Jia Xin\AppData\Local\Temp\ipykernel_3324\914188826.py:10: DtypeWarning: Columns (0: CPI wts: Non-energy industrial goods GOODS, 1: CPI wts: Durables GOODS, 2: CPI wts: Semi-durables GOODS, 3: CPI wts: Non-durables GOODS, 4: CPI wts: Non-seasonal food GOODS, 5: CPI wts: Food, alcoholic beverages & tobacco GOODS, 6: CPI wts: Processed food & non-alcoholic beverages GOODS, 7: CPI wts: Unprocessed food GOODS, 8: CPI wts: Seasonal food GOODS, 9: CPI wts: Industrial goods GOODS, 10: CPI wts: Energy GOODS, 11: CPI wts: Electricity, gas & misc. energy GOODS, 12: CPI wts: CPI excluding tobacco SPECIAL AGGREGATES, 13: CPI wts: Clothing & footwear goods GOODS, 14: CPI wts: Housing goods GOODS, 15: CPI wts: Household goods GOODS, 16: CPI wts: Water supply; materials for maintenance & repair GOODS, 17: CPI wts: Medical goods GOODS, 18: CPI wts: Vehicles, spare parts & accessories GOODS, 19: CPI wts: Recreational goods GOODS, 20: CPI wts: Audio-visual goods GOODS, 21: CPI wts: Other recrea

Selected by CDID: D7CY -> CPI INDEX 12.1 : PERSONAL CARE 2015=100


('../data/processed/cpi_personal_care.csv',
 (457, 4),
 {'source_file': '../data/raw/ons_cpi_mm23.csv',
  'series_title': 'CPI INDEX 12.1 : PERSONAL CARE 2015=100',
  'cdid': 'D7CY',
  'unit': 'Index, base year = 100',
  'release_date': '18-02-2026',
  'base_year': '2015=100',
  'rows': 457,
  'date_min': '1988-01-01',
  'date_max': '2026-01-01'})